In [4]:
#!/usr/bin/env python3
"""
Leak-free Cross-Validated CCA between ViT embeddings and mouse brain activity.

Pipeline (per fold):
  1) Split images into TRAIN/TEST.
  2) On TRAIN ONLY:
       - PCA on ViT (images × D_vit) to 90% variance (cap by n_train-1).
       - PCA on Brain (images × neurons) to 90% variance (cap by n_train-1).
       - Center transformed TRAIN sets.
       - Fit CCA with n_components = min(CCA_MAX, r_vit, r_brain, n_train-1).
  3) Project TEST into the TRAIN-fitted PCA spaces, center with TRAIN means,
     transform with the TRAIN-fitted CCA, and compute canonical correlations.
  4) (Optional) Permutation test on TEST: shuffle image pairing between views
     and recompute the first canonical correlation to form an empirical p-value.

Saves:
  - <prefix>_cca_folds.csv          (per-fold, per-component train/test corr)
  - <prefix>_cca_summary.csv        (mean±std across folds)
  - <prefix>_cca_perm_pvals.csv     (optional permutation p for comp-1 per fold)

Author: Maria + Pläku 🐾
"""

import numpy as np
import pandas as pd
import pickle
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold
from sklearn.cross_decomposition import CCA
from scipy.special import softmax
from skbio.stats.composition import clr
from numpy.random import default_rng

# -----------------------------
# CONFIG
# -----------------------------
VIT_PATH    = '/home/maria/Documents/HuggingMouseData/MouseViTEmbeddings/google_vit-base-patch16-224_embeddings_logits.pkl'
NEURAL_PATH = '/home/maria/LuckyMouse/pixel_transformer_neuro/data/processed/hybrid_neural_responses.npy'
AREAS_PATH  = '/home/maria/MITNeuralComputation/visualization/brain_area.npy'

AREA_NAME   = 'VISp'     # change to VISam / VISp / etc.
N_IMAGES, N_TRIALS = 118, 50

N_SPLITS    = 2           # 2-fold for stronger test power
RANDOM_STATE= 42
CCA_MAX     = 10          # hard cap on # canonical dims
RUN_PERM    = True        # set False to skip permutation test
N_PERM      = 1000        # permutations for test-set empirical p (comp-1)
OUT_PREFIX  = f'vit_{AREA_NAME}'

# -----------------------------
# UTILITIES
# -----------------------------
def center_train_apply_test(X_tr, X_te):
    """Center columns using TRAIN means; return centered copies and means."""
    mu = X_tr.mean(axis=0, keepdims=True)
    return X_tr - mu, X_te - mu, mu

def train_pca(X_train, var_cut=0.90, seed=RANDOM_STATE):
    """Fit PCA on TRAIN; choose n_components to reach var_cut, cap at rank."""
    pca_full = PCA(random_state=seed).fit(X_train)
    cum = np.cumsum(pca_full.explained_variance_ratio_)
    n90 = int(np.searchsorted(cum, var_cut) + 1)
    rank_cap = min(X_train.shape[0]-1, X_train.shape[1])  # ≤ n_train-1
    ncomp = max(1, min(n90, rank_cap))
    pca = PCA(n_components=ncomp, random_state=seed).fit(X_train)
    return pca

def canonical_corrs(U, V):
    """Return vector of canonical correlations between columns of U and V."""
    K = min(U.shape[1], V.shape[1])
    cs = []
    for k in range(K):
        u = U[:, k]; v = V[:, k]
        # protect against degenerate columns
        su = u.std(ddof=1); sv = v.std(ddof=1)
        if su == 0 or sv == 0: 
            cs.append(np.nan)
        else:
            cs.append(np.corrcoef(u, v)[0,1])
    return np.array(cs)

# -----------------------------
# LOAD DATA
# -----------------------------
print("🔹 Loading ViT embeddings...")
with open(VIT_PATH, 'rb') as f:
    vit_logits = pickle.load(f)['natural_scenes']   # (images × D_vit)
vit_soft = softmax(np.asarray(vit_logits), axis=1)
vit_clr  = clr(vit_soft + 1e-12)                   # (images × D_vit)

print(f"🔹 Loading neural responses ({AREA_NAME})...")
dat   = np.load(NEURAL_PATH, mmap_mode='r')
areas = np.load(AREAS_PATH, allow_pickle=True)
mask  = areas == AREA_NAME
dat   = dat[mask]
n_neurons, n_total = dat.shape
n_time = n_total // (N_IMAGES * N_TRIALS)
dat = dat.reshape(n_neurons, N_IMAGES, N_TRIALS, n_time)
X_mean = dat.mean(axis=(2,3))                      # (neurons × images)
brain_img_by_neuron = X_mean.T                     # (images × neurons)

print(f"{AREA_NAME}: {n_neurons} neurons × {N_IMAGES} images")

# -----------------------------
# K-FOLD CV
# -----------------------------
kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=RANDOM_STATE)
fold_rows = []
perm_rows = []
rng = default_rng(RANDOM_STATE)

for fold_id, (train_idx, test_idx) in enumerate(kf.split(np.arange(N_IMAGES)), 1):
    print(f"\n==== Fold {fold_id} ====")
    n_train, n_test = len(train_idx), len(test_idx)
    print(f"Train images: {n_train}, Test images: {n_test}")

    Xv_tr, Xv_te = vit_clr[train_idx], vit_clr[test_idx]                  # ViT
    Xb_tr, Xb_te = brain_img_by_neuron[train_idx], brain_img_by_neuron[test_idx]  # Brain

    # --- PCA on TRAIN ONLY (both sides) ---
    pca_v = train_pca(Xv_tr)
    pca_b = train_pca(Xb_tr)
    Zv_tr = pca_v.transform(Xv_tr)     # (n_train × r_v)
    Zv_te = pca_v.transform(Xv_te)     # (n_test  × r_v)
    Zb_tr = pca_b.transform(Xb_tr)     # (n_train × r_b)
    Zb_te = pca_b.transform(Xb_te)     # (n_test  × r_b)

    # --- Center TRAIN features; apply same centering to TEST ---
    Zv_tr_c, Zv_te_c, _ = center_train_apply_test(Zv_tr, Zv_te)
    Zb_tr_c, Zb_te_c, _ = center_train_apply_test(Zb_tr, Zb_te)

    # --- CCA on TRAIN ---
    n_comp_cca = min(CCA_MAX, Zv_tr_c.shape[1], Zb_tr_c.shape[1], n_train - 1)
    n_comp_cca = max(1, n_comp_cca)
    print(f"  PCA dims: ViT={Zv_tr_c.shape[1]}, Brain={Zb_tr_c.shape[1]}  |  CCA components={n_comp_cca}")

    cca = CCA(n_components=n_comp_cca, max_iter=1000, scale=False)
    cca.fit(Zv_tr_c, Zb_tr_c)

    # Canonical variates / correlations: TRAIN
    U_tr, V_tr = cca.transform(Zv_tr_c, Zb_tr_c)
    cc_tr = canonical_corrs(U_tr, V_tr)

    # Canonical variates / correlations: TEST (project using TRAIN fit)
    U_te, V_te = cca.transform(Zv_te_c, Zb_te_c)
    cc_te = canonical_corrs(U_te, V_te)

    for k in range(n_comp_cca):
        fold_rows.append({
            "fold": fold_id,
            "component": k+1,
            "train_corr": cc_tr[k],
            "test_corr":  cc_te[k],
            "n_train": n_train,
            "n_test":  n_test,
            "vit_pca_dim": Zv_tr_c.shape[1],
            "brain_pca_dim": Zb_tr_c.shape[1]
        })

    # --- Optional: permutation test on TEST for comp-1 ---
    if RUN_PERM and n_test >= 10:
        # Shuffle the pairing between the two views on TEST by permuting brain rows
        obs = cc_te[0]
        perm_vals = []
        for _ in range(N_PERM):
            perm = rng.permutation(n_test)
            U_p, V_p = cca.transform(Zv_te_c, Zb_te_c[perm])
            r_p = canonical_corrs(U_p, V_p)[0]
            perm_vals.append(r_p if np.isfinite(r_p) else 0.0)
        perm_vals = np.asarray(perm_vals)
        # two-sided empirical p
        p_emp = (np.sum(np.abs(perm_vals) >= np.abs(obs)) + 1) / (N_PERM + 1)
        perm_rows.append({
            "fold": fold_id,
            "component": 1,
            "test_corr": float(obs),
            "perm_p_two_sided": float(p_emp),
            "n_perm": N_PERM
        })
        print(f"  Permutation test (comp-1): r_test={obs:+.3f}, p_emp={p_emp:.4f}")

# -----------------------------
# SAVE RESULTS
# -----------------------------
fold_df = pd.DataFrame(fold_rows)
fold_df.to_csv(f"{OUT_PREFIX}_cca_folds.csv", index=False)
print(f"\n💾 Saved per-fold CCA results -> {OUT_PREFIX}_cca_folds.csv")

# Summary across folds per component index
if not fold_df.empty:
    summ = (fold_df
            .groupby("component")
            .agg(train_corr_mean=("train_corr","mean"),
                 train_corr_std =("train_corr","std"),
                 test_corr_mean =("test_corr","mean"),
                 test_corr_std  =("test_corr","std"),
                 n_folds=("fold","nunique"))
            .reset_index()
            .sort_values("component"))
    summ.to_csv(f"{OUT_PREFIX}_cca_summary.csv", index=False)
    print(f"💾 Saved summary -> {OUT_PREFIX}_cca_summary.csv")
    print("\n===== CCA summary (mean ± std across folds) =====")
    for _,row in summ.iterrows():
        print(f"  comp {int(row.component):2d} | "
              f"train {row.train_corr_mean:+.3f} ± {row.train_corr_std:.3f}  |  "
              f"test {row.test_corr_mean:+.3f} ± {row.test_corr_std:.3f}")

# Permutation p-values
if RUN_PERM and len(perm_rows):
    perm_df = pd.DataFrame(perm_rows)
    perm_df.to_csv(f"{OUT_PREFIX}_cca_perm_pvals.csv", index=False)
    print(f"💾 Saved permutation p-values (comp-1) -> {OUT_PREFIX}_cca_perm_pvals.csv")


🔹 Loading ViT embeddings...
🔹 Loading neural responses (VISp)...
VISp: 14382 neurons × 118 images

==== Fold 1 ====
Train images: 59, Test images: 59
  PCA dims: ViT=30, Brain=45  |  CCA components=10
  Permutation test (comp-1): r_test=+0.324, p_emp=0.0190

==== Fold 2 ====
Train images: 59, Test images: 59
  PCA dims: ViT=29, Brain=44  |  CCA components=10
  Permutation test (comp-1): r_test=+0.323, p_emp=0.0100

💾 Saved per-fold CCA results -> vit_VISp_cca_folds.csv
💾 Saved summary -> vit_VISp_cca_summary.csv

===== CCA summary (mean ± std across folds) =====
  comp  1 | train +1.000 ± 0.000  |  test +0.323 ± 0.001
  comp  2 | train +0.963 ± 0.002  |  test +0.189 ± 0.089
  comp  3 | train +0.948 ± 0.005  |  test +0.306 ± 0.129
  comp  4 | train +0.958 ± 0.059  |  test +0.289 ± 0.051
  comp  5 | train +0.967 ± 0.046  |  test +0.102 ± 0.042
  comp  6 | train +0.913 ± 0.123  |  test +0.061 ± 0.065
  comp  7 | train +0.928 ± 0.102  |  test +0.083 ± 0.253
  comp  8 | train +0.895 ± 0.148